# Introduction

In this notebook I will be exploring the text structure of Reddit whisky reviews stored at 

# Imports/Setup

In [1]:
import os
import pandas as pd
import numpy as np
import pymongo
import spacy
import nltk
from nltk.corpus import stopwords as STOPWORDS
import textacy

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.p

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!sudo python -m spacy download en
nlp = spacy.load('en')

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /usr/local/anaconda/lib/python3.6/site-packages/en_core_web_sm -->
    /

In [4]:
def textacy_process(string):
    return textacy.preprocess_text(string,lowercase=True, no_urls=True, no_emails=True,\
                        no_phone_numbers=True,no_numbers=False,no_currency_symbols=False,no_punct=True,\
                        no_contractions=True,no_accents=True)

# Data Load

In [5]:
test_data = pd.read_csv('/mnt/data/processed/Reddit_Whisky_Network_Review_Archive_20190424_wuuid.csv')

In [6]:
test_data.head()

,Unnamed: 0,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid
0,0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12,54e266b4abda4e278055f0a3254b0ef2
1,1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18,18b5c00c30a244a49bfa15132192417c
2,2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17,780c90af30b7447090fbc1fc1c857ecf
3,3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18,f86e6c62559743b4a20d85d0fb2594e8
4,4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018,3bc513dc90d944439df8b2f6805b7edf


# WhiskyReviews Database Connection

In [7]:
client = pymongo.MongoClient("mongodb://whiskyadmin:%s@whiskyreviews-shard-00-00-mvfds.mongodb.net:27017,whiskyreviews-shard-00-01-mvfds.mongodb.net:27017,whiskyreviews-shard-00-02-mvfds.mongodb.net:27017/test?ssl=true&replicaSet=WhiskyReviews-shard-0&authSource=admin&retryWrites=true" % (os.getenv('mongopw')))
reddit_reviews = client.reddit_reviews

In [8]:
reddit_reviews.list_collection_names()

['submissions']

In [9]:
submissions = reddit_reviews['submissions']

In [10]:
documents = [x for x in submissions.find()]

In [11]:
len(documents)

31128

In [12]:
documents[0]

{'_id': '54e266b4abda4e278055f0a3254b0ef2',
 'submission_id': '14uder',
 'first_comment': "My wife and I are on a trip to Thailand to meet her family.  I've seen plenty of whisky here, mostly JW, but this one stood out from the rest.  100 pipers is not something I've seen before and it seems to have quite the following here.  It is a blend at 40% alcohol by volume and 35cl was 220 baht or about $8 Canadian.  I got it more as a novelty as I suspect it is the Thai equivalent of chivas or glenfiddich 12.\n\nColour: caramel, I suspect it is artificially coloured.\n\nNose: (I had some tiger balm on my hands so this may be *way* off) alcohol, little bit of leather and some hints of sweetness.\n\nPalate: very bland, I taste almost nothing really, a bit of woody flavour, the promise of leather and sweetness from the nose is gone.\n\nFinish: short and devoid of anything but alcohol.\n\nThis reminds me of a JW red or the cheap rye my Dad drank when I was a kid.  I bought it primarily for the nov

In [13]:
reviews = pd.DataFrame(documents)

In [14]:
reviews.replace("", np.nan, inplace=True)

In [15]:
reviews['bestguess_review'] = reviews.selftext.combine_first(reviews.first_comment)

In [16]:
def nose_test(str):
    if str is None:
        return False
    else:
        return 'nose' in str.lower()
    
def palate_test(str):
    if str is None:
        return False
    else:
        return 'palate' in str.lower() or 'taste' in str.lower()
    
def finish_test(str):
    if str is None:
        return False
    else:
        return 'finish' in str.lower()

In [17]:
reviews['nose_flag'] = reviews.bestguess_review.apply(nose_test)

In [18]:
reviews['palate_flag'] = reviews.bestguess_review.apply(palate_test)

In [19]:
reviews['finish_flag'] = reviews.bestguess_review.apply(finish_test)

In [20]:
reviews['nose_flag'].value_counts()

True     28409
False     2719
Name: nose_flag, dtype: int64

In [21]:
reviews['palate_flag'].value_counts()

True     28639
False     2489
Name: palate_flag, dtype: int64

In [22]:
reviews['finish_flag'].value_counts()

True     28622
False     2506
Name: finish_flag, dtype: int64

In [23]:
reviews.iloc[144].first_comment

"I've heard good things about Aberfeldy 21 OB. Seems like a very standard, no bells and whistles whisky."

In [24]:
reviews.iloc[13].selftext

'Mystery Sample picked and poured by my wife. Tasting notes messaged to /u/mzoltek for reveal.        \n&nbsp;   \n\n**Color** :   \n                                                               \n**Nose** : Roasted coffee, burnt butter & oak char.  \n    \n**Taste** : Deep oak notes, black coffee, bitter dark chocolate, brown sugar, tons of caramel, eucalyptus, wintergreen, peppermint & slight smoke.  \n  \n      \n**Finish** : Long and oaky, medium mouthfeel.  \n\n&nbsp;  \n\n\n**Guess** : 120-125 proof, 6-8 years, CEHT Barrel Proof?  \n \n**Reveal** : [1792 Full Proof 125 Proof](/spoiler)    \n\n**Conclusion** :/u/flavorjunkie most definitely needs to do a SBS on this with ECBP. **3/5**  '

In [25]:
print(reviews.loc[0].bestguess_review)

My wife and I are on a trip to Thailand to meet her family.  I've seen plenty of whisky here, mostly JW, but this one stood out from the rest.  100 pipers is not something I've seen before and it seems to have quite the following here.  It is a blend at 40% alcohol by volume and 35cl was 220 baht or about $8 Canadian.  I got it more as a novelty as I suspect it is the Thai equivalent of chivas or glenfiddich 12.

Colour: caramel, I suspect it is artificially coloured.

Nose: (I had some tiger balm on my hands so this may be *way* off) alcohol, little bit of leather and some hints of sweetness.

Palate: very bland, I taste almost nothing really, a bit of woody flavour, the promise of leather and sweetness from the nose is gone.

Finish: short and devoid of anything but alcohol.

This reminds me of a JW red or the cheap rye my Dad drank when I was a kid.  I bought it primarily for the novelty so I don't think it was a waste.  it is just not something I'd seek out again.

68/100

PS: Yes 

# Parsing

Now that we have a basic idea of how populated these views are, let's pull all documents, parse the review text with spacy, and store them back into MongoDB for later use. 

In [26]:
stop_words = set(STOPWORDS.words('english'))

In [27]:
review_text = reviews.bestguess_review.tolist()

In [28]:
reviews.head()

,_id,created_utc,first_comment,permalink,selftext,submission_id,subreddit,title,bestguess_review,nose_flag,palate_flag,finish_flag
0,54e266b4abda4e278055f0a3254b0ef2,1.355495e+09,My wife and I are on a trip to Thailand to mee...,/r/Scotch/comments/14uder/100_pipers_blend_rev...,NaN,14uder,r/Scotch,100 Pipers Blend Review #10,My wife and I are on a trip to Thailand to mee...,True,True,True
1,18b5c00c30a244a49bfa15132192417c,1.540938e+09,The 1792 series continues and this time we are...,/r/bourbon/comments/9ssqyk/review_35_1792_225t...,NaN,9ssqyk,r/bourbon,Review #35: 1792 225th Anniversary,The 1792 series continues and this time we are...,True,True,True
2,780c90af30b7447090fbc1fc1c857ecf,1.507072e+09,Bourbon Review #48 1792 225th Anniversary \n\n...,/r/bourbon/comments/7445vx/bourbon_review_48_1...,NaN,7445vx,r/bourbon,Bourbon Review #48 - 1792 225th Anniversary,Bourbon Review #48 1792 225th Anniversary \n\n...,True,True,True
3,f86e6c62559743b4a20d85d0fb2594e8,1.540681e+09,The 1792 review series continues! The Bottled ...,/r/bourbon/comments/9ryi8f/review_34_1792_bott...,NaN,9ryi8f,r/bourbon,Review #34: 1792 Bottled in Bond,The 1792 review series continues! The Bottled ...,True,True,True
4,3bc513dc90d944439df8b2f6805b7edf,1.545201e+09,This week Oak Liquor Cabinet got in the long a...,/r/bourbon/comments/a7jwdd/reviews_5153_oak_li...,NaN,a7jwdd,r/bourbon,Reviews #51-53: Oak Liquor Cabinet 1792 Picks,This week Oak Liquor Cabinet got in the long a...,True,True,True


In [29]:
_ = [stop_words.add(x) for x in ['email','phone','url']]

### Create a New Collection in MongoDB

In [30]:
parsed_docs = reddit_reviews['parsed']

In [31]:
errors = []
i = 0

for i, row in enumerate(reviews.itertuples()):
    try:
        textacy_parsed = textacy_process(row.bestguess_review)
        parsed = nlp(textacy_parsed)
        tokens = [token.lower_ for token in parsed if not ((token.lower_ in stop_words) or token.is_punct or token.is_space or token.lemma_ == '-PRON-')]
        lemmas = [token.lemma_ for token in parsed if not ((token.lower_ in stop_words) or token.is_punct or token.is_space or token.lemma_ == '-PRON-')]
        #sents = [sent for sent in parsed.sents]
        parsed_doc = {}
        parsed_doc['_id'] = row._1
        parsed_doc['tokens'] = tokens
        parsed_doc['lemmas'] = lemmas
        #parsed_doc['sents'] = sents
        # We can't store off the PRAW object since it can't be encoded.

        parsed_docs.insert_one(parsed_doc)
    except:
        errors.append(parsed_doc)
        
    if i%1000 == 0:
        print('iteration ', i, 'with ', len(errors), 'errors')
    
    i += 1


iteration  0 with  0 errors
iteration  1000 with  0 errors
iteration  2000 with  0 errors
iteration  3000 with  0 errors
iteration  4000 with  0 errors
iteration  5000 with  0 errors
iteration  6000 with  0 errors
iteration  7000 with  0 errors
iteration  8000 with  0 errors
iteration  9000 with  0 errors
iteration  10000 with  0 errors
iteration  11000 with  0 errors
iteration  12000 with  0 errors
iteration  13000 with  0 errors
iteration  14000 with  0 errors
iteration  15000 with  0 errors
iteration  16000 with  0 errors
iteration  17000 with  0 errors
iteration  18000 with  0 errors
iteration  19000 with  0 errors
iteration  20000 with  0 errors
iteration  21000 with  0 errors
iteration  22000 with  0 errors
iteration  23000 with  0 errors
iteration  24000 with  0 errors
iteration  25000 with  1 errors
iteration  26000 with  1 errors
iteration  27000 with  1 errors
iteration  28000 with  1 errors
iteration  29000 with  2 errors
iteration  30000 with  2 errors
iteration  31000 with

In [33]:
errors

[{'_id': 'a7a2d3ac218c4b78917b89a2ded96eaf',
  'tokens': ['hi',
   'quick',
   'review',
   'blended',
   'malt',
   'tonight',
   'courtesy',
   'grants',
   'wanting',
   'try',
   'one',
   'really',
   'decently',
   'priced',
   'trader',
   'joe',
   'california',
   'mere',
   '$',
   '29',
   '99',
   'could',
   'potentially',
   'become',
   'daily',
   'speysider',
   'satisfies',
   'blend',
   'glenfiddich',
   'balvenie',
   'kinivie',
   'single',
   'malts',
   'u',
   'dworgi',
   'pointed',
   'review',
   'cheers',
   'monkey',
   'shoulder',
   'batch',
   '27',
   'blended',
   'malt',
   '40',
   'abv',
   'chill',
   'filtered',
   'e150a',
   '$',
   '29',
   '99',
   'enjoyed',
   'neat',
   'jpg',
   'color',
   'amber',
   'nose',
   'fruity',
   'red',
   'apples',
   'faint',
   'vanilla',
   'soft',
   'pinot',
   'noir',
   'note',
   'rich',
   'tropical',
   'fruity',
   'note',
   'wet',
   'oak',
   'light',
   'nutmeg',
   'decent',
   'nose',
   're

In [34]:
parsed_docs.insert_one(errors[0])

DuplicateKeyError: E11000 duplicate key error collection: reddit_reviews.parsed index: _id_ dup key: { : "a7a2d3ac218c4b78917b89a2ded96eaf" }

In [35]:
parsed_docs.insert_one(errors[1])

DuplicateKeyError: E11000 duplicate key error collection: reddit_reviews.parsed index: _id_ dup key: { : "613c454ed9454159936d02e96e28afaf" }